# * Pipeline de Preprocesamiento y Análisis Exploratorio (EDA)
**Proyecto:** Análisis Socioeconómico de Los Ángeles  
**Autores:** Equipo de Ciencia de Datos  
**Fecha:** 2025-XX-XX

Este notebook sirve como base para el preprocesamiento de los tres datasets principales:
- Datos de crimen
- Datos de negocios
- Datos de precios de vivienda (ZHVI)

Cada integrante del equipo podrá trabajar con su dataset, aplicando los mismos pasos de limpieza y análisis exploratorio.  
Se utilizará el módulo `01_eda_preprocessing.py` para realizar los pasos de forma modular y reutilizable.

In [ ]:
# Importación de librerías necesarias

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import geopandas as gpd  # type: ignore
from shapely.geometry import Point  # type: ignore
import requests
import io
from http.client import IncompleteRead

# Importar módulo de funciones de preprocesamiento
import sys
sys.path.append('./notebooks')  # Ajustar si es necesario
import eda_preprocessing as eda  # type: ignore

# Configuración de estilo
sns.set_style("darkgrid")
pd.options.display.float_format = "{:,.2f}".format

In [ ]:
# Función para descargar datasets por streaming

def fetch_csv_stream(url: str, timeout: int = 60) -> io.StringIO:
    """
    Descarga un archivo CSV utilizando streaming.
    Maneja errores por lectura incompleta (IncompleteRead).
    """
    try:
        resp = requests.get(url, stream=True, timeout=timeout)
        resp.raise_for_status()
        text = resp.text
    except IncompleteRead as e:
        print('Lectura incompleta:', e.partial)
        text = e.partial.decode('utf-8', errors='replace')
    return io.StringIO(text)


In [ ]:
# ======================================
# Dataset de Crimen – Los Ángeles (2020–presente)
# ======================================

# URL oficial del dataset
crime_url = "https://data.lacity.org/api/views/2nrs-mtv8/rows.csv?accessType=DOWNLOAD"

# Descarga y lectura del archivo
crime_buffer = fetch_csv_stream(crime_url)
crime_df = pd.read_csv(crime_buffer, low_memory=False)
del crime_buffer  # Liberar memoria

# Limpieza de nombres de columnas
crime_df.columns = crime_df.columns.str.strip()

# Conversión de columnas de fecha
crime_df["Date Rptd"] = pd.to_datetime(crime_df["Date Rptd"], format="%m/%d/%Y %I:%M:%S %p", errors="coerce")
crime_df["DATE OCC"] = pd.to_datetime(crime_df["DATE OCC"], format="%m/%d/%Y %I:%M:%S %p", errors="coerce")

# Vista rápida
crime_df.head(3)


In [ ]:
# ======================================
# Dataset de Negocios – Los Ángeles
# ======================================

# URL oficial del dataset
business_url = "https://data.lacity.org/api/views/r4uk-afju/rows.csv?accessType=DOWNLOAD"

# Descarga y lectura del archivo
business_buffer = fetch_csv_stream(business_url)
business_df = pd.read_csv(business_buffer, low_memory=False)
del business_buffer

# Limpieza de nombres de columnas
business_df.columns = business_df.columns.str.strip()

# Conversión de columnas de fecha
business_df["LOCATION START DATE"] = pd.to_datetime(business_df["LOCATION START DATE"], format="%m/%d/%Y", errors="coerce")
business_df["LOCATION END DATE"] = pd.to_datetime(business_df["LOCATION END DATE"], format="%m/%d/%Y", errors="coerce")

# Vista rápida
business_df.head(3)

In [ ]:
# ======================================
# Dataset de Vivienda – Índice ZHVI de Zillow
# ======================================

# Ruta al archivo local descargado desde Zillow
zhvi_path = "./data/Neighborhood_zhvi.csv"  # ⚠️ Asegúrate de tenerlo en esta ruta

# Carga del archivo
zhvi_df = pd.read_csv(zhvi_path, low_memory=False)

# Filtrar solo registros de la ciudad de Los Ángeles (CA)
zhvi_df_la = zhvi_df[(zhvi_df["State"] == "CA") & (zhvi_df["City"] == "Los Angeles")]

# Eliminar columnas innecesarias para el análisis
zhvi_df_la = zhvi_df_la.drop(["RegionType", "StateName", "State", "City", "Metro", "CountyName"], axis=1)

# Convertir a formato largo (long format)
id_cols = zhvi_df_la.columns[:3]
zhvi_long = zhvi_df_la.melt(id_vars=id_cols, var_name="date", value_name="Price")

# Conversión de fechas + columnas de año y mes
zhvi_long["date"] = pd.to_datetime(zhvi_long["date"], errors="coerce")
zhvi_long["year"] = zhvi_long["date"].dt.year
zhvi_long["month"] = zhvi_long["date"].dt.month

# Vista rápida
zhvi_long.head(3)

